## Setup

Requirements:

- Python 3.8
- CPLEX Studio 20.1.0
- docplex library configured - only works with Python 3.8

## Import dependencies

In [1]:
# namedtuple creates a tuple with a name
# https://docs.python.org/3/library/collections.html#collections.namedtuple
from collections import namedtuple

# Model: https://ibmdecisionoptimization.github.io/docplex-doc/mp/docplex.mp.model.html
# Model is a class to embed modeling objects
from docplex.mp.model import Model
# LinearExpr contains the negate() function to create negate a linear expression
from docplex.mp.linear import LinearExpr

# used for normalisation of vectors
import numpy as np

# get all possible combinations for staticLex
import itertools

In [2]:
def optimize_offensive_army(axe_strength = 1, lc_strength = 1, ma_strength = 1, build_time_limit_seconds = 2_592_000, log_output = False): 
    # Initialise data

    Unit = namedtuple('Unit', ['name', 'att_strength', 'recruit_time_in_seconds', 'food'])

    axe_unit = Unit('axe', 45, 90, 1)
    lc_unit = Unit('lc', 130, 360, 4)
    ma_unit = Unit('ma', 150, 450, 5)
    serk_unit = Unit('serk', 300, 1200, 6)
    ram_unit = Unit('ram', 2, 480, 5)

    units = [axe_unit, lc_unit, ma_unit, serk_unit, ram_unit]
    barracks_units = [axe_unit, lc_unit, ma_unit, ram_unit]
    hoo_units = [serk_unit]

    with Model(name='Max strength unit time') as m:
        # Decision variable
        number_of_units = m.integer_var_dict(
            [u.name for u in units], name = 'number_of_units'
        )
        number_of_units

        # Decision expressions
        m.barracks_build_time = m.sum([number_of_units[u.name] * u.recruit_time_in_seconds for u in barracks_units])
        m.hoo_build_time = m.sum([number_of_units[u.name] * u.recruit_time_in_seconds for u in hoo_units])

        m.totalNegativeAttack = LinearExpr.negate(m.sum([number_of_units[u.name] * u.att_strength for u in units]))
        m.totalFood = m.sum([number_of_units[u.name] * u.food for u in units])

        # Constraints
        ct_food_no_church = m.add_constraints([m.totalFood <= 20596, m.totalFood >= 20580])
        ct_must_have_rams = m.add_constraint(number_of_units['ram'] >= 250)

        ct_build_time_less_than_4_weeks_barracks = m.add_constraints([1 <= m.barracks_build_time, (m.barracks_build_time <= build_time_limit_seconds)])
        ct_build_time_less_than_4_weeks_hoo = m.add_constraints([1 <= m.hoo_build_time, (m.hoo_build_time <= build_time_limit_seconds)])

        attack_strength_weights = [axe_strength, lc_strength, ma_strength]
        strength_ratios = attack_strength_weights / np.sum(attack_strength_weights)

        # Provide a range for the divided strength constraints
        lower_bound_cap = 0.95

        ct_axe_number = m.add_constraints([
            number_of_units['axe'] * axe_unit.att_strength <= -(m.totalNegativeAttack * strength_ratios[0]),
            number_of_units['axe'] * axe_unit.att_strength >= -(m.totalNegativeAttack * strength_ratios[0]) * lower_bound_cap
        ])

        ct_axe_number = m.add_constraints([
            number_of_units['lc'] * lc_unit.att_strength <= -(m.totalNegativeAttack * strength_ratios[1]),
            number_of_units['lc'] * lc_unit.att_strength >= -(m.totalNegativeAttack * strength_ratios[1]) * lower_bound_cap
        ])

        ct_axe_number = m.add_constraints([
            number_of_units['ma'] * ma_unit.att_strength <= -(m.totalNegativeAttack * strength_ratios[2]),
            number_of_units['ma'] * ma_unit.att_strength >= -(m.totalNegativeAttack * strength_ratios[2]) * lower_bound_cap
        ])

        # Objectives
        m.add_kpi(m.totalNegativeAttack, "Total negative attack strength")
        m.add_kpi(m.barracks_build_time, "Barracks build time")
        m.add_kpi(m.hoo_build_time, "Hall of Order build time")

        # Initialise KPI permutations
        kpis_permutations = itertools.permutations([m.totalNegativeAttack, m.hoo_build_time, m.barracks_build_time], 3)
        kpis_permutations = list(kpis_permutations)

        if log_output is True:
            # pretty names for results
            kpis_permutations_names = []
            for permutation in kpis_permutations:
                name = []
                for kpi in permutation:
                    if kpi is m.totalNegativeAttack:
                        name.append('totalNegativeAttack')
                    if kpi is m.hoo_build_time:
                        name.append('hoo_build_time')
                    if kpi is m.barracks_build_time:
                        name.append('barracks_build_time')
                kpis_permutations_names.append(name)

        # Solve
        solutions = []
        for idx, kpis in enumerate(kpis_permutations):
            m.minimize_static_lex(kpis)
            m.solve()

            solutions.append({
                "axe": number_of_units['axe'].solution_value,
                "lc": number_of_units['lc'].solution_value,
                "ma": number_of_units['ma'].solution_value,
                "serk": number_of_units['serk'].solution_value,
                "ram": number_of_units['ram'].solution_value,
                "food": sum([number_of_units[u.name].solution_value * u.food for u in units]),
                "time_in_seconds": sum([number_of_units[u.name].solution_value * u.recruit_time_in_seconds for u in units]),
                "time_in_days": round(sum([number_of_units[u.name].solution_value * u.recruit_time_in_seconds for u in units]) / (24 * 3600), 2)
            })

            if log_output is True:
                print(f"{kpis_permutations_names[idx]} - strength: {-m.totalNegativeAttack.solution_value}, food: {m.totalFood.solution_value}")
                print(
                    f"axe: {number_of_units['axe'].solution_value}"
                    f", lc: {number_of_units['lc'].solution_value}"
                    f", ma: {number_of_units['ma'].solution_value}"
                    f", serk: {number_of_units['serk'].solution_value}"
                    f", ram: {number_of_units['ram'].solution_value}"
                    f", food: {sum([number_of_units[u.name].solution_value * u.food for u in units])}"
                    f", time: {round(sum([number_of_units[u.name].solution_value * u.recruit_time_in_seconds for u in units]) / (24 * 3600), 2)} days"
                )
                print(' ')
        
        return solutions

In [3]:
# no parameters, assume equal strengths, 30 days in a month build time cap, no log outputs
optimize_offensive_army()

[{'axe': 4814.0,
  'lc': 1664.0,
  'ma': 1442.0,
  'serk': 111.0,
  'ram': 250.0,
  'food': 20596.0,
  'time_in_seconds': 1934400.0,
  'time_in_days': 22.39},
 {'axe': 4814.0,
  'lc': 1664.0,
  'ma': 1442.0,
  'serk': 111.0,
  'ram': 250.0,
  'food': 20596.0,
  'time_in_seconds': 1934400.0,
  'time_in_days': 22.39},
 {'axe': 4985.0,
  'lc': 1725.0,
  'ma': 1491.0,
  'serk': 1.0,
  'ram': 250.0,
  'food': 20596.0,
  'time_in_seconds': 1861800.0,
  'time_in_days': 21.55},
 {'axe': 4979.0,
  'lc': 1720.0,
  'ma': 1493.0,
  'serk': 1.0,
  'ram': 250.0,
  'food': 20580.0,
  'time_in_seconds': 1860360.0,
  'time_in_days': 21.53},
 {'axe': 4807.0,
  'lc': 1663.0,
  'ma': 1441.0,
  'serk': 111.0,
  'ram': 250.0,
  'food': 20580.0,
  'time_in_seconds': 1932960.0,
  'time_in_days': 22.37},
 {'axe': 4807.0,
  'lc': 1663.0,
  'ma': 1441.0,
  'serk': 111.0,
  'ram': 250.0,
  'food': 20580.0,
  'time_in_seconds': 1932960.0,
  'time_in_days': 22.37}]

In [4]:
# no parameters, playstyle caters to calvary-heavy build
optimize_offensive_army(axe_strength = 3, lc_strength = 6, ma_strength = 9)

[{'axe': 2263.0,
  'lc': 1567.0,
  'ma': 2037.0,
  'serk': 105.0,
  'ram': 250.0,
  'food': 20596.0,
  'time_in_seconds': 1930440.0,
  'time_in_days': 22.34},
 {'axe': 2263.0,
  'lc': 1567.0,
  'ma': 2037.0,
  'serk': 105.0,
  'ram': 250.0,
  'food': 20596.0,
  'time_in_seconds': 1930440.0,
  'time_in_days': 22.34},
 {'axe': 2341.0,
  'lc': 1621.0,
  'ma': 2103.0,
  'serk': 1.0,
  'ram': 250.0,
  'food': 20596.0,
  'time_in_seconds': 1861800.0,
  'time_in_days': 21.55},
 {'axe': 2339.0,
  'lc': 1620.0,
  'ma': 2101.0,
  'serk': 1.0,
  'ram': 250.0,
  'food': 20580.0,
  'time_in_seconds': 1860360.0,
  'time_in_days': 21.53},
 {'axe': 2261.0,
  'lc': 1566.0,
  'ma': 2035.0,
  'serk': 105.0,
  'ram': 250.0,
  'food': 20580.0,
  'time_in_seconds': 1929000.0,
  'time_in_days': 22.33},
 {'axe': 2261.0,
  'lc': 1566.0,
  'ma': 2035.0,
  'serk': 105.0,
  'ram': 250.0,
  'food': 20580.0,
  'time_in_seconds': 1929000.0,
  'time_in_days': 22.33}]